In [112]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

import torch.nn.functional as F

#plotting
import matplotlib.pyplot as plt

In [113]:
import data_utils
import models
import importlib
import transformer_models
importlib.reload(data_utils)
importlib.reload(models)
importlib.reload(transformer_models)
from data_utils import *
from models import *
from transformer_models import * 
import torch.optim as optim
from torch.utils.data import DataLoader
import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [114]:
data_dir = "data/"
dataObject = xrdData("data/", device)

In [115]:
dataObject.make_datasets(1, composition_embedding="composition1D")
# Create DataLoaders for train and validation sets
train_loader = DataLoader(dataObject.torch_datasets['train'], batch_size=256, shuffle=True)
valid_loader = DataLoader(dataObject.torch_datasets['val'], batch_size=256, shuffle=False)  

In [137]:
# Create the model instance and move it to the selected device
output_dim = 230  # Output dimension
ConvModel = XRD_C_SymNet(in_channels=1, output_dim=output_dim, composition_model= None).to(device)
token_size = 50 #dimension of the tokens 
TransModel = TransformerModel(ntoken = output_dim, d_model = token_size, nhead = 10, d_hid=50, nlayers=1, dropout = 0.5).to(device)
model = ConvModel
# Define optimizer and loss function
weight_decay = 0  # Example value, adjust based on your needs
optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

12160


In [138]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [139]:
count_parameters(ConvModel)

31369900

In [140]:
count_parameters(TransModel)

148027400

In [141]:
count_parameters(ConvModel) / count_parameters(TransModel)

0.21191955002925134

In [142]:
#instantiate experimental data simulator 
simulator = ExperimentalSimulation(device) 

In [143]:
max_epochs = 200
metrics = ["accuracy", "loss"]

log = {
    f"{type}": {f"{metric}" : np.zeros(max_epochs) for metric in metrics} for type in ['train', 'val']     
}

for epoch in range(max_epochs):
    model.train()  # Set the model to training mode
    total_train_loss = 0
    correct_train = 0
    total_train = 0

    for xrd, composition, targets in train_loader: 
        xrd = F.normalize(xrd, p=2, dim=1)
        optimizer.zero_grad()
        #xrd = simulator.sim(xrd)
        if model == TransModel:
            outputs = model(xrd) #add composition support later 
        else:
            outputs = model(xrd, composition)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()

    train_accuracy = 100 * correct_train / total_train
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_valid_loss = 0
    correct_valid = 0
    total_valid = 0

    with torch.no_grad():  # No gradients needed for validation
        for xrd, composition, targets in valid_loader:
            xrd = F.normalize(xrd, p=2, dim=1)
            #xrd = simulator.sim(xrd)
            if model == TransModel:
                outputs = model(xrd) #add composition support later 
            else:
                outputs = model(xrd, composition)
            loss = criterion(outputs, targets)
            total_valid_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1) 
            total_valid += targets.size(0)
            correct_valid += (predicted == targets).sum().item()

    valid_accuracy = 100 * correct_valid / total_valid

    total_train_loss = total_train_loss / len(train_loader)
    validation_loss = total_valid_loss / len(valid_loader)

    print(f"Epoch {epoch+1}, Training Loss: {total_train_loss}, Training Accuracy: {train_accuracy}%, Validation Loss: {validation_loss}, Validation Accuracy: {valid_accuracy}%")

    log['train']['accuracy'][epoch] = (train_accuracy)
    log['train']['loss'][epoch] =(total_train_loss)

    log['val']['accuracy'][epoch] =(valid_accuracy)
    log['val']['loss'][epoch] =(validation_loss)

Epoch 1, Training Loss: 3.166233497308501, Training Accuracy: 28.303616063989335%, Validation Loss: 2.4453689787122936, Validation Accuracy: 40.99049303559585%
Epoch 2, Training Loss: 2.3027346184913147, Training Accuracy: 44.6675554074321%, Validation Loss: 2.0642301771375866, Validation Accuracy: 49.17090426707937%
Epoch 3, Training Loss: 1.940155248269967, Training Accuracy: 52.069099594512025%, Validation Loss: 1.7931274440553453, Validation Accuracy: 54.963519787751494%
Epoch 4, Training Loss: 1.6914627053213458, Training Accuracy: 57.20713214464256%, Validation Loss: 1.6237857275538974, Validation Accuracy: 58.235684280344906%
Epoch 5, Training Loss: 1.5088031444143741, Training Accuracy: 60.96761650835972%, Validation Loss: 1.570229967435201, Validation Accuracy: 60.09285872208711%
Epoch 6, Training Loss: 1.3534991047906537, Training Accuracy: 64.26428928511915%, Validation Loss: 1.417560186651018, Validation Accuracy: 63.54189697103692%


KeyboardInterrupt: 

In [ ]:
model_dir = "models/"

date = datetime.now()
model_name = f"first_model_{date.month}_{date.day}"

os.makedirs(os.path.join(model_dir, model_name), exist_ok=True)

torch.save(model, os.path.join(model_dir, model_name, 'model.pth'))

for data_type, metrics_dict in log.items():
    for metric, array in metrics_dict.items():
        filename = f"{data_type}_{metric}.npy"  # Construct filename, e.g., "train_accuracy.npy"
        filename = os.path.join(model_dir, model_name, filename)
        np.save(filename, array)  # Save the array to a file

AttributeError: module 'datetime' has no attribute 'now'